<a href="https://colab.research.google.com/github/midori142/devc-innovation-challenge/blob/master/Assignment_3_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [0]:
# Import pandas, numpy
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


In [26]:
# Read dataset with extra params sep='\t', encoding="latin-1"
train = pd.read_csv('labeledTrainData.csv', sep='\t', encoding="latin-1")
train

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary BuÃ±uel made in the...


## 2. Preprocessing

In [34]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
print(stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'each', 'about', 'd', 'more', "should've", "isn't", 'is', "hasn't", 'was', 'over', 'but', "haven't", 'are', 'before', 'wasn', 'not', 'been', "shouldn't", 'this', 'myself', 'for', 'off', "you've", 'needn', 'isn', 'those', 'can', 'any', 'had', 'll', 'there', "weren't", "wouldn't", 'i', "you'd", 'whom', 'same', "don't", "needn't", 'again', 'yours', 'her', 'now', 'most', 'while', 'hers', 'him', 'what', 'the', 'as', 't', 'why', "mightn't", "won't", 'm', 'own', 'yourself', 'ain', 'we', 'when', 'shan', 'ma', 'than', 'on', 'because', 'you', 'through', 'theirs', 'mightn', 'who', 'that', 'they', 'these', 'above', 'some', 'only', 'so', 'will', 'just', 'until', 'am', 'doing', "shan't", 'here', 'out', 'its', 'won', 'y', 'has', 'nor', "doesn't", 'his', 'with', "that'll", 'hasn', 'during', 'aren', 'wouldn', 'up', 'no', 's', 'an', 'by', 'of', 'didn', 'from', 'in', 'don', "wasn't", 't

In [0]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emo = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emo).replace('-', ''))
    return text

In [0]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer
# Your code here

porter=PorterStemmer()

def tokenizer_porter(text):
    token = []
    token = [porter.stem(word) for word in text.split()]
    return token # Your code here

In [0]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X=train['review']
Y=train['sentiment']
x_train, x_test, y_train,y_test  = train_test_split(X,Y, test_size = 0.2)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [40]:
# Import Pipeline, LogisticRegression, TfidfVectorizer

tfidf  = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7fa75b08ce18>,
                                 smooth_idf=True,
                                 stop_words={'a', 'about', 'above', 'aft...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7fa75b13e620>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
          

## 4. Evaluate Model

In [41]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
y_hat=np.array(y_test)
y_predict = clf.predict(x_test)
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

accuracy_score(y_hat, y_predict)
classification_report(y_hat, y_predict).split('\n')


['              precision    recall  f1-score   support',
 '',
 '           0       0.90      0.87      0.89      2472',
 '           1       0.88      0.91      0.89      2528',
 '',
 '    accuracy                           0.89      5000',
 '   macro avg       0.89      0.89      0.89      5000',
 'weighted avg       0.89      0.89      0.89      5000',
 '']

In [45]:
print(pd.DataFrame(confusion_matrix(y_hat, y_predict)))


      0     1
0  2149   323
1   230  2298


## 5. Export Model 

In [0]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('LR.pkl'), 'wb'), protocol=4)
